In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "virny-flow-experiments":
    os.chdir("../../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/Research/NYU/VirnyFlow_Project/Code/virny-flow-experiments


In [4]:
import pandas as pd
from duckdb import query as sqldf
from virny_flow.core.custom_classes.core_db_client import CoreDBClient
from virny_flow.configs.constants import EXP_CONFIG_HISTORY_TABLE, ALL_EXPERIMENT_METRICS_TABLE
from virny_flow.visualizations.use_case_queries import get_models_disparity_metric_df
from source.visualizations.use_case_queries import get_best_lps_per_exp_config

## Prepare data for visualizations

In [5]:
SECRETS_PATH = os.path.join(os.getcwd(), "scripts", "configs", "secrets.env")
TARGET_DATASET = 'folk_pubcov'

In [6]:
VIRNY_FLOW = 'virny_flow'
ALPINE = 'alpine_meadow'
AUTOSKLEARN = 'autosklearn'
DATASET_TO_METRICS = {
    'diabetes': [
        {'F1': 'overall'},
        {'Label_Stability': 'overall'},
    ],
    'folk_emp': [
        {'F1': 'overall'},
        {'Equalized_Odds_FNR': 'SEX&RAC1P'},
    ],
    'folk_pubcov': [
        {'F1': 'overall'},
        {'Selection_Rate_Difference': 'SEX'},
        {'Selection_Rate_Difference': 'RAC1P'},
    ],
    'heart': [
        {'F1': 'overall'},
        {'Equalized_Odds_TNR': 'gender'},
    ],
}
EXP_CONFIG_NAMES = {
    VIRNY_FLOW: {
        'diabetes': 'comparison_exp2_diabetes_w32_vf',
        'folk_emp': 'comparison_exp2_folk_emp_w32_vf',
        'folk_pubcov': 'comparison_exp2_folk_pubcov_w32_vf',
        'heart': 'comparison_exp2_heart_w32_vf',
    },
    ALPINE: {
        'diabetes': 'comparison_exp2_diabetes_w32_am',
        'folk_emp': 'comparison_exp2_folk_emp_w32_am',
        'folk_pubcov': 'comparison_exp2_folk_pubcov_w32_am',
        'heart': 'comparison_exp2_heart_w32_am',
    },
    AUTOSKLEARN: {
        'diabetes': 'comparison_exp2_diabetes_w32_askl_workers_32',
        'folk_emp': 'comparison_exp2_folk_emp_w32_askl_workers_32',
        'folk_pubcov': 'comparison_exp2_folk_pubcov_w32_askl_workers_32',
        'heart': 'comparison_exp2_heart_w32_askl_workers_32',
    },
}

db_client = CoreDBClient(SECRETS_PATH)
db_client.connect()

In [7]:
def get_virny_flow_metrics(db_client):
    exp_config_names = [EXP_CONFIG_NAMES['virny_flow'][TARGET_DATASET]]
    best_lp_metrics_per_exp_config_df = get_best_lps_per_exp_config(secrets_path=SECRETS_PATH,
                                                                    exp_config_names=exp_config_names)
    virny_flow_all_runtime_df = pd.DataFrame()
    for exp_config_name in exp_config_names:
        virny_flow_runtime_df = db_client.read_metric_df_from_db(collection_name=EXP_CONFIG_HISTORY_TABLE,
                                                                 query={'exp_config_name': exp_config_name,
                                                                        'deletion_flag': False})
        virny_flow_all_runtime_df = pd.concat([virny_flow_all_runtime_df, virny_flow_runtime_df])
    
    new_column_names = []
    for col in virny_flow_all_runtime_df.columns:
        new_col_name = '_'.join([c.lower() for c in col.split('_')])
        new_column_names.append(new_col_name)
    virny_flow_all_runtime_df.columns = new_column_names

    virny_flow_metrics_df = sqldf("""
        SELECT DISTINCT t1.*, t2.exp_config_execution_time
        FROM best_lp_metrics_per_exp_config_df AS t1
        JOIN virny_flow_all_runtime_df AS t2
          ON t1.exp_config_name = t2.exp_config_name
         AND t1.run_num = t2.run_num
    """).to_df()
    
    return virny_flow_metrics_df


def get_system_metrics(db_client, system_name: str):
    exp_config_names = [EXP_CONFIG_NAMES[system_name][TARGET_DATASET]]
    system_all_runtime_df = pd.DataFrame()
    for exp_config_name in exp_config_names:
        system_runtime_df = db_client.read_metric_df_from_db(collection_name=ALL_EXPERIMENT_METRICS_TABLE,
                                                                 query={'exp_config_name': exp_config_name,
                                                                        'system_name': system_name,
                                                                        'tag': 'OK'})
        
        new_column_names = []
        for col in system_runtime_df.columns:
            new_col_name = '_'.join([c.lower() for c in col.split('_')])
            new_column_names.append(new_col_name)
        system_runtime_df.columns = new_column_names
    
        # Create columns based on values in the Subgroup column
        pivoted_all_metrics_df = pd.DataFrame()
        for run_num in system_runtime_df['run_num'].unique():
            subset_df = system_runtime_df[system_runtime_df['run_num'] == run_num]
            pivoted_metrics_df = subset_df.pivot(columns='subgroup', values='metric_value',
                                                 index=[col for col in subset_df.columns
                                                        if col not in ('subgroup', 'metric_value')]).reset_index()
            pivoted_all_metrics_df = pd.concat([pivoted_all_metrics_df, pivoted_metrics_df])
        
        system_all_runtime_df = pd.concat([system_all_runtime_df, pivoted_all_metrics_df])

    return system_all_runtime_df

In [8]:
virny_flow_metrics_df = get_virny_flow_metrics(db_client)

Extracting metrics for comparison_exp2_folk_pubcov_w32_vf...
best_pps_per_lp_and_run_num_df.shape: (99, 19)
best_lp_per_run_all.shape: (90, 19)
Extracted metrics for comparison_exp2_folk_pubcov_w32_vf



In [9]:
alpine_meadow_metrics_df = get_system_metrics(db_client, system_name=ALPINE)

In [10]:
autosklearn_metrics_df = get_system_metrics(db_client, system_name=AUTOSKLEARN)

## Display Results

In [11]:
def display_table_with_many_metrics(system_metrics_df, system_name: str, metrics_lst: list, dataset_name: str):
    if system_name == VIRNY_FLOW:
        system_metrics_df['system_name'] = system_name
        common_cols = ['system_name', 'dataset_name', 'run_num', 'exp_config_execution_time']
    else:
        common_cols = ['system_name', 'dataset_name', 'run_num', 'optimization_time']

    f1_metrics_df = system_metrics_df[system_metrics_df['metric'] == 'F1']
    f1_metrics_df['F1'] = f1_metrics_df['overall']
    f1_metrics_df = f1_metrics_df[common_cols + ['F1']]
    
    if dataset_name == "diabetes":
        ls_metrics_df = system_metrics_df[system_metrics_df['metric'] == 'Label_Stability']
        ls_metrics_df['Label_Stability'] = ls_metrics_df['overall']
        ls_metrics_df = ls_metrics_df[common_cols + ['Label_Stability']]

        final_metrics_df = sqldf(f"""
            SELECT t1.*, t2.Label_Stability
            FROM f1_metrics_df AS t1
            JOIN ls_metrics_df AS t2
              ON t1.run_num = t2.run_num
             AND t1.dataset_name = t2.dataset_name
        """).to_df()
    elif len(metrics_lst) == 3:
        disparity_metric_name1, group_name1 = list(metrics_lst[1].items())[0]
        disparity_metric_name2, group_name2 = list(metrics_lst[2].items())[0]
        
        disparity_metric_df1 = get_models_disparity_metric_df(system_metrics_df, disparity_metric_name1, group_name1)
        disparity_metric_df1[disparity_metric_name1] = disparity_metric_df1['disparity_metric_value']
        disparity_metric_df1 = disparity_metric_df1[common_cols + [disparity_metric_name1]]
        
        disparity_metric_df2 = get_models_disparity_metric_df(system_metrics_df, disparity_metric_name2, group_name2)
        disparity_metric_df2[disparity_metric_name2] = disparity_metric_df2['disparity_metric_value']
        disparity_metric_df2 = disparity_metric_df2[common_cols + [disparity_metric_name2]]

        final_metrics_df = sqldf(f"""
            SELECT t1.*, t2.{disparity_metric_name1} AS {disparity_metric_name1}_{group_name1}, t3.{disparity_metric_name2} AS {disparity_metric_name2}_{group_name2}
            FROM f1_metrics_df AS t1
            JOIN disparity_metric_df1 AS t2
              ON t1.run_num = t2.run_num
             AND t1.dataset_name = t2.dataset_name
            JOIN disparity_metric_df2 AS t3
              ON t1.run_num = t3.run_num
             AND t1.dataset_name = t3.dataset_name
        """).to_df()
    else:
        disparity_metric_name, group_name = list(metrics_lst[1].items())[0]
        disparity_metric_df = get_models_disparity_metric_df(system_metrics_df, disparity_metric_name, group_name)
        disparity_metric_df[disparity_metric_name] = disparity_metric_df['disparity_metric_value']
        disparity_metric_df = disparity_metric_df[common_cols + [disparity_metric_name]]
    
        final_metrics_df = sqldf(f"""
            SELECT t1.*, t2.{disparity_metric_name}
            FROM f1_metrics_df AS t1
            JOIN disparity_metric_df AS t2
              ON t1.run_num = t2.run_num
             AND t1.dataset_name = t2.dataset_name
        """).to_df()

    if system_name == VIRNY_FLOW:
        final_metrics_df = final_metrics_df[~final_metrics_df['exp_config_execution_time'].isna()]
        final_metrics_df = final_metrics_df.rename(columns={'exp_config_execution_time': 'optimization_time'})

    return final_metrics_df


def display_table_with_results_for_dataset(virny_flow_metrics_df: pd.DataFrame, alpine_meadow_metrics_df: pd.DataFrame, 
                                           autosklearn_metrics_df: pd.DataFrame, dataset_name: str):
    metrics_lst = DATASET_TO_METRICS[dataset_name]
    virny_flow_final_metrics_df = display_table_with_many_metrics(virny_flow_metrics_df[virny_flow_metrics_df['dataset_name'] == dataset_name], VIRNY_FLOW, metrics_lst, dataset_name)
    alpine_meadow_final_metrics_df = display_table_with_many_metrics(alpine_meadow_metrics_df[alpine_meadow_metrics_df['dataset_name'] == dataset_name], ALPINE, metrics_lst, dataset_name)
    autosklearn_final_metrics_df = display_table_with_many_metrics(autosklearn_metrics_df[autosklearn_metrics_df['dataset_name'] == dataset_name], AUTOSKLEARN, metrics_lst, dataset_name)

    final_metrics_df = pd.concat([virny_flow_final_metrics_df, alpine_meadow_final_metrics_df, autosklearn_final_metrics_df])
    return final_metrics_df


def create_latex_table(df):
    # Compute mean and std for each system
    summary = df.groupby('system_name').agg(['mean', 'std']).round(3)
    
    # Combine mean and std into "mean ± std" format
    def format_metric(mean, std):
        if pd.isna(std):
            return f"{mean:.3f} ± n/a"
        return f"{mean:.3f} \scriptsize{{$\pm${std:.3f}}}"
    
    # Create formatted DataFrame
    disparity_metric = list(DATASET_TO_METRICS[TARGET_DATASET][1].keys())[0]
    if TARGET_DATASET == "folk_pubcov":
        group = list(DATASET_TO_METRICS[TARGET_DATASET][1].values())[0]
        disparity_metric2, group2 = list(DATASET_TO_METRICS[TARGET_DATASET][2].items())[0]
        disparity_metric += '_' + group
        disparity_metric2 += '_' + group2
        latex_df = pd.DataFrame({
            'System': summary.index,
            'F1': [format_metric(m, s) for m, s in zip(summary['F1']['mean'], summary['F1']['std'])],
            disparity_metric: [format_metric(m, s) for m, s in zip(summary[disparity_metric]['mean'], summary[disparity_metric]['std'])],
            disparity_metric2: [format_metric(m, s) for m, s in zip(summary[disparity_metric2]['mean'], summary[disparity_metric2]['std'])],
        })
    else:
        latex_df = pd.DataFrame({
            'System': summary.index,
            'F1': [format_metric(m, s) for m, s in zip(summary['F1']['mean'], summary['F1']['std'])],
            disparity_metric: [format_metric(m, s) for m, s in zip(summary[disparity_metric]['mean'], summary[disparity_metric]['std'])]
        })
    
    # Reorder rows: virny_flow first
    latex_df = latex_df.set_index('System').loc[['virny_flow', 'alpine_meadow', 'autosklearn']].reset_index()
    
    # Generate LaTeX table
    latex_table = latex_df.to_latex(index=False,
                                    caption='Performance Comparison by System',
                                    label='tab:performance_summary',
                                    column_format='lcc',
                                    escape=False)
    
    print(latex_table)

In [12]:
results_df = display_table_with_results_for_dataset(virny_flow_metrics_df, alpine_meadow_metrics_df, autosklearn_metrics_df, dataset_name=TARGET_DATASET)

In [13]:
results_df

,system_name,dataset_name,run_num,optimization_time,F1,Selection_Rate_Difference_SEX,Selection_Rate_Difference_RAC1P
0,virny_flow,folk_pubcov,8,3617.460723,0.624408,-0.008430,0.095064
1,virny_flow,folk_pubcov,1,3609.941209,0.625256,-0.012055,0.090666
2,virny_flow,folk_pubcov,3,3601.582368,0.555641,-0.011151,0.027681
3,virny_flow,folk_pubcov,2,3625.397271,0.564645,-0.000082,0.043105
4,virny_flow,folk_pubcov,6,3622.947461,0.611487,-0.014736,0.081335
5,virny_flow,folk_pubcov,9,3600.705347,0.612581,-0.011851,0.089005
6,virny_flow,folk_pubcov,10,3606.599357,0.574293,-0.002246,0.045188
7,virny_flow,folk_pubcov,4,3612.060215,0.556766,-0.010052,0.026560
8,virny_flow,folk_pubcov,5,3608.324075,0.619939,-0.021716,0.092757
9,virny_flow,folk_pubcov,7,3604.934581,0.614494,-0.021631,0.091113


In [14]:
create_latex_table(results_df)

\begin{table}
\centering
\caption{Performance Comparison by System}
\label{tab:performance_summary}
\begin{tabular}{lcc}
\toprule
       System &                            F1 &  Selection_Rate_Difference_SEX & Selection_Rate_Difference_RAC1P \\
\midrule
   virny_flow & 0.596 \scriptsize{$\pm$0.029} & -0.011 \scriptsize{$\pm$0.007} &   0.068 \scriptsize{$\pm$0.029} \\
alpine_meadow & 0.628 \scriptsize{$\pm$0.006} & -0.033 \scriptsize{$\pm$0.008} &   0.178 \scriptsize{$\pm$0.011} \\
  autosklearn & 0.633 \scriptsize{$\pm$0.009} & -0.033 \scriptsize{$\pm$0.010} &   0.177 \scriptsize{$\pm$0.008} \\
\bottomrule
\end{tabular}
\end{table}

